In [1]:
import pandas as pd
import json
import os, re
import ast
import numpy as np

# Read ground truth annotations for object-attribute relations

In [2]:
attributes = pd.read_csv('../gold_dataset/gold_attributes_relations_500pts_500studies1st.txt',sep='\t')
print(attributes.shape)
# attributes['region'] = attributes.groupby(['image_id','row_id'])['bbox'].transform(lambda x: '&&'.join(x))
# attributes['region'] = [sorted(list(set(x.split('&&')))) for x in attributes['region']]
print(len(set(attributes.row_id)))
attributes.head()

(21594, 14)
3042


,patient_id,study_id,image_id,row_id,section,bbox,relation,label_name,context,categoryID,region,annot_id,sentence,object_id
0,10020740,55522869,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484.dcm,55522869|5,finalreport,carina,1,endotracheal tube,yes,tubesandlines,"['carina', 'neck', 'trachea']",55522869|5|carina|1|endotracheal tube,There is an endotracheal tube in place with it...,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484_c...
1,10020740,55522869,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484.dcm,55522869|5,finalreport,neck,1,endotracheal tube,yes,tubesandlines,"['carina', 'neck', 'trachea']",55522869|5|neck|1|endotracheal tube,There is an endotracheal tube in place with it...,NaN
2,10020740,55522869,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484.dcm,55522869|5,finalreport,trachea,1,endotracheal tube,yes,tubesandlines,"['carina', 'neck', 'trachea']",55522869|5|trachea|1|endotracheal tube,There is an endotracheal tube in place with it...,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484_t...
3,10020740,55522869,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484.dcm,55522869|6,finalreport,abdomen,1,enteric tube,yes,tubesandlines,"['abdomen', 'mediastinum', 'neck']",55522869|6|abdomen|1|enteric tube,Nasogastric tube extends well into the stomach.,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484_a...
4,10020740,55522869,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484.dcm,55522869|6,finalreport,mediastinum,1,enteric tube,yes,tubesandlines,"['abdomen', 'mediastinum', 'neck']",55522869|6|mediastinum|1|enteric tube,Nasogastric tube extends well into the stomach.,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484_m...


In [3]:
print('Number of unique report studies annotated:',len(set(attributes.study_id)))
print('From number of unique patients:',len(set(attributes.patient_id)))

Number of unique report studies annotated: 500
From number of unique patients: 500


In [4]:
uniq_rel = sorted(list(set(['|'.join(x.split('|')[2:]) for x in attributes.annot_id])))
print('There are ', len(uniq_rel), ' types of unique object-to-attribute relations')
pos_rel = [x for x in uniq_rel if '|1|' in x]
neg_rel = [x for x in uniq_rel if '|0|' in x]
print('Of which, there are: ')
print(len(pos_rel), ' affirmed relation types')
print(len(neg_rel), ' negated relation types')

# negated relations are extracted from sentences that specifically describe some attribute NOT being at the bbox location in the image
# affirmed relations are extracted from sentences that describe the attribute being or probable at the bbox location in the image

# Device and tubes and lines are also treated as attributes in the Chest ImaGenome dataset
# The object-to-attribute relations, if affirmed, should be more interpretted as the device is "in" or "near by" the bbox location(s)
# More NLP work needs to be done to determine the exact relation between the device and the locations 
# e.g. is the endotracheal above or below the carina?
# We plan to extract these sort of relations from creating questions and answer pairs from the current annotations


There are  619  types of unique object-to-attribute relations
Of which, there are: 
436  affirmed relation types
183  negated relation types


In [5]:
# These are the anatomical locations evaluated for the NLP pipeline
print(len(set(attributes.bbox)))
print(set(attributes.bbox))

38
{'upper mediastinum', 'right shoulder', 'right mid lung zone', 'left hilar structures', 'cavoatrial junction', 'trachea', 'mediastinum', 'left clavicle', 'right breast', 'right arm', 'right atrium', 'left breast', 'right hilar structures', 'cardiac silhouette', 'right apical zone', 'neck', 'right chest wall', 'left lung', 'svc', 'left hemidiaphragm', 'abdomen', 'aortic arch', 'left arm', 'right upper lung zone', 'right hemidiaphragm', 'right costophrenic angle', 'right clavicle', 'left apical zone', 'carina', 'left chest wall', 'left lower lung zone', 'right lung', 'left upper lung zone', 'left mid lung zone', 'left costophrenic angle', 'spine', 'left shoulder', 'right lower lung zone'}


In [6]:
# These are the attributes evaluated for the NLP pipeline
# Even though we ran over 90 different attributes for the whole Chest ImaGenome dataset
# only these 70 are found in the 500 reports annotated -- we scoured all sentences from each report for recall
print(len(set(attributes.label_name)))
print(set(attributes.label_name))

70
{'atelectasis', 'alveolar hemorrhage', 'costophrenic angle blunting', 'calcified nodule', 'shoulder osteoarthritis', 'mediastinal drain', 'hernia', 'intra-aortic balloon pump', 'breast/nipple shadows', 'pericardial effusion', 'vascular redistribution', 'pulmonary edema/hazy opacity', 'endotracheal tube', 'tortuous aorta', 'lung cancer', 'pneumomediastinum', 'lobar/segmental collapse', 'pleural effusion', 'enteric tube', 'aspiration', 'sub-diaphragmatic air', 'pneumonia', 'mass/nodule (not otherwise specified)', 'subclavian line', 'scoliosis', 'vascular calcification', 'increased reticular markings/ild pattern', 'spinal degenerative changes', 'subcutaneous air', 'hydropneumothorax', 'lung lesion', 'mediastinal widening', 'elevated hemidiaphragm', 'superior mediastinal mass/enlargement', 'fluid overload/heart failure', 'cyst/bullae', 'linear/patchy atelectasis', 'chest port', 'bone lesion', 'consolidation', 'normal', 'granulomatous disease', 'enlarged hilum', 'interstitial lung diseas

In [7]:
# These are a subset of the above that have bbox coordinates in the Chest ImaGenome dataset
bboxlist = ['aortic arch', 'svc', 'trachea', 'carina', 'upper mediastinum'
            , 'cavoatrial junction', 'right atrium', 'cardiac silhouette'
            , 'mediastinum'
            , 'left apical zone', 'left upper lung zone', 'left mid lung zone', 'left lower lung zone', 'left lung'
            , 'right apical zone' , 'right upper lung zone', 'right mid lung zone', 'right lower lung zone', 'right lung'
            , 'left clavicle', 'right clavicle'
            , 'left hilar structures', 'right hilar structures'
            , 'left costophrenic angle', 'right costophrenic angle'
            , 'left hemidiaphragm', 'right hemidiaphragm'
            ,'abdomen', 'spine'
            ]

In [8]:
# These are the anatomical locations with NLP derived object-attribute relations but do not have bbox coordinates
print(set(attributes.bbox).difference(set(bboxlist)))

## However, these anatomies can be approximately derived on a CXR image:
# right chest wall can be approximately derived from slightly enlarging the right lung bbox, vice versa for left
# right breast usually would overlap in the right lower lung zone region -- where nipples can be mistaken for nodules -- similarly for left
# right shoulder approximately overlap the region encompassed by the right clavicle and right upper lung zone, and vice versa for left
# right arm is approximately between right edge of image to the right of the right lung, and vice versa for left
# neck would be above the clavicles to the top of the image, following the trachea bbox but would be wider.

## Note these derivation rules are subject to patient positioning and quality of CXR projection for different images.
## Best to semi automatically generate some ground truth and train an object detection network to recognize them more reliably.


{'left chest wall', 'left breast', 'right shoulder', 'left arm', 'neck', 'right chest wall', 'left shoulder', 'right breast', 'right arm'}


# Get all first studies for the 500 patients

In [9]:
# Read in the file with all the raw information for the gold standard dataset
gold = pd.read_csv('../gold_dataset/gold_all_sentences_500pts_1000studies.txt',sep='\t')
print('Number of unique patients:',len(set(gold.patient_id)))
print('Number of unique studies:',len(set(gold.study_id)))

# first CXR exam for each patient to assess for object-attribute relation extraction
imageIDs1 = gold[gold.StudyOrder == 1][['image_id']].drop_duplicates().image_id.tolist()
print('Number of first exam images', len(imageIDs1))

Number of unique patients: 500
Number of unique studies: 1000
Number of first exam images 500


In [10]:
gold.head()

,patient_id,image_id,StudyOrder,study_id,sent_loc,row_id,section,sentence
0,10020740,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484.dcm,1,55522869,1.0,55522869|1,finalreport,FINAL REPORT\n EXAMINATION: CHEST (PORTABLE AP)
1,10020740,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484.dcm,1,55522869,2.0,55522869|2,history,INDICATION: ___ year old man with h/o acute ...
2,10020740,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484.dcm,1,55522869,3.0,55522869|3,finalreport,IMPRESSION:
3,10020740,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484.dcm,1,55522869,4.0,55522869|4,finalreport,No previous images.
4,10020740,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484.dcm,1,55522869,5.0,55522869|5,finalreport,There is an endotracheal tube in place with it...


# Read automatically extracted scene graphs

In [11]:
def readJSON(filepath):
    try:
        with open(filepath) as f:
            data = json.load(f)
            return data
    except Exception as e:
        print('File does not exist',filepath)
        return None
        
# Read relevant parts for the scene graph json    
def readSceneGraph(image_id, basepath):
    filepath = os.path.join(basepath,str(image_id.replace('.dcm','')) + '_SceneGraph.json')
    data = readJSON(filepath)  
    objects = dict()
    objs = data['objects']
    for obj in objs:
        key = obj['object_id']
        objects[key] = obj
    attributes = dict()
    attrs = data['attributes']
    for attr in attrs:
        key = attr['bbox_name']
        if attr[key]:
            attributes[key] = attr
    relations = data['relationships']
    return objects, attributes, relations


#   Used to generate the raw nlp results file
def graph2tab_direct(imageIDs, basepath):
    tab = {'image_id':[],'study_id':[],'sent_loc':[],'row_id':[],'bbox':[],'attribute':[]
           ,'context':[],'label_name':[],'categoryID':[], 'sentence':[]
           ,'comparison':[],'temporal':[],'laterality':[],'severity':[],'texture':[]
          }
    for image_id in imageIDs:
        if os.path.isfile(os.path.join(basepath,str(image_id.replace('.dcm','')) + '_SceneGraph.json')):
            objects, attributes, relations = readSceneGraph(image_id, basepath)
            for key, attri in attributes.items():
                #sents = attri['sentences']
                sents = attri['phrases']
                if sents != None:
                    phraseIDs = attri['phrase_IDs']
                    attributes = attri['attributes']
                    comparisons = attri['comparison_cues']
                    for sent, phrase_id, att, comp in zip(sents,phraseIDs,attributes,comparisons):
                        comp = [x for x in comp if x in ['comparison|yes|worsened','comparison|yes|improved','comparison|yes|no change']]
                        # 'texture', 'anatomicalfinding', 'nlp', 'device',  'technicalassessment', 'tubesandlines', 'disease'
                        # 'temporal', 'laterality', 'severity'
                        sentloc = phrase_id.split('|')[1]
                        study_id = phrase_id.split('|')[0]
                        row_id = study_id+'|'+sentloc
                        if len(att)>0:
                            temporal = [x for x in att if 'temporal|' in x]
                            laterality = [x for x in att if 'laterality|' in x]
                            severity = [x for x in att if 'severity|' in x]
                            texture = [x for x in att if 'texture|' in x]
                            att2 = [x for x in att if not any(cat in x for cat in ['temporal','laterality','severity','texture'])]
                            for a in att2:
                                if a != '':
                                    tab['image_id'].append(image_id)
                                    tab['study_id'].append(study_id)
                                    tab['sent_loc'].append(sentloc)
                                    tab['row_id'].append(row_id)
                                    tab['bbox'].append(key)
                                    tab['attribute'].append(a)
                                    tab['context'].append(a.split('|')[1])
                                    tab['label_name'].append(a.split('|')[2])
                                    tab['categoryID'].append(a.split('|')[0])
                                    tab['sentence'].append(sent)
                                    tab['comparison'].append(str(comp))
                                    tab['temporal'].append(str(temporal))
                                    tab['laterality'].append(str(laterality))
                                    tab['severity'].append(str(severity))
                                    tab['texture'].append(str(texture))
        else:
                print('File does not exist', image_id.replace('.dcm','') + '_SceneGraph.json')                                
    dfnlp = pd.DataFrame(tab)
    print(dfnlp.shape)
    dfnlp = dfnlp.drop_duplicates().reset_index(drop=True).copy()
    print(dfnlp.shape)
    
    # Get region by sentence info -- all the areas describe or implied by the findings described in the sentence
    dfnlp['region'] = dfnlp.groupby(['image_id','row_id'])['bbox'].transform(lambda x: '&&'.join(x))
    dfnlp['region'] = [sorted(list(set(x.split('&&')))) for x in dfnlp['region']]
    
    # Get annotation column that can be compared with the ground truth
    dfnlp['relation'] = [1 if x =='yes' else 0 for x in dfnlp['context']]
    dfnlp['annot_id'] = ['|'.join([str(row_id),bbox,str(rela),lab]) if lab !='' else '' for row_id, lab, rela, bbox in zip(dfnlp['row_id'],dfnlp['label_name'],dfnlp['relation'],dfnlp['bbox'])]
    print(dfnlp.shape)
        
    return dfnlp

In [12]:
# TO DO: set path to the scene_graph json directory
images_dir = '../../../subset/scene_graph/' 

In [13]:
# Turn scene graph to table for easier comparison against ground truth
scene = graph2tab_direct(imageIDs1, images_dir)

# Turn study_id from string to int
scene['study_id'] = [int(x) for x in scene['study_id']]

(21882, 15)
(21882, 15)
(21882, 18)


In [14]:
scene.head()

,image_id,study_id,sent_loc,row_id,bbox,attribute,context,label_name,categoryID,sentence,comparison,temporal,laterality,severity,texture,region,relation,annot_id
0,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484.dcm,55522869,9,55522869|9,right lung,anatomicalfinding|yes|lung opacity,yes,lung opacity,anatomicalfinding,There may well be a small right pleural effusion.,[],[],[],[],[],"[right costophrenic angle, right lung]",1,55522869|9|right lung|1|lung opacity
1,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484.dcm,55522869,9,55522869|9,right lung,anatomicalfinding|yes|pleural effusion,yes,pleural effusion,anatomicalfinding,There may well be a small right pleural effusion.,[],[],[],[],[],"[right costophrenic angle, right lung]",1,55522869|9|right lung|1|pleural effusion
2,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484.dcm,55522869,9,55522869|9,right lung,nlp|yes|abnormal,yes,abnormal,nlp,There may well be a small right pleural effusion.,[],[],[],[],[],"[right costophrenic angle, right lung]",1,55522869|9|right lung|1|abnormal
3,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484.dcm,55522869,8,55522869|8,right lung,anatomicalfinding|no|vascular congestion,no,vascular congestion,anatomicalfinding,Mild\n basilar atelectatic changes without evi...,[],[],[],[],[],"[left hilar structures, left lower lung zone, ...",0,55522869|8|right lung|0|vascular congestion
4,27776756-1d9ef4fc-cd8dd0ca-1453072f-12c0f484.dcm,55522869,8,55522869|8,right lung,anatomicalfinding|yes|atelectasis,yes,atelectasis,anatomicalfinding,Mild\n basilar atelectatic changes without evi...,[],[],[],[],[],"[left hilar structures, left lower lung zone, ...",1,55522869|8|right lung|1|atelectasis


In [15]:
# Making sure we're evaluating over same set of study_ids, attributes and objects
print(scene.shape)
scene_attr = scene[(scene.study_id.isin(attributes.study_id))&(scene.attribute!='')].reset_index(drop=True).copy()
print(scene_attr.shape)
scene_attr = scene_attr[(scene_attr.label_name.isin(attributes.label_name))&(scene_attr.bbox.isin(attributes.bbox))].reset_index(drop=True).copy()
print(scene_attr.shape)

(21882, 18)
(21882, 18)
(21882, 18)


# Calculate sentence level results

In [16]:
sentlevel_labs = list(set(['|'.join(x.split('|')[2:]) for x in attributes['annot_id']]))
sentlevel_labs.sort()
print('Number of unique bbox to attribute annotations:', len(sentlevel_labs))
for lab in sentlevel_labs:
    print(lab)

Number of unique bbox to attribute annotations: 619
abdomen|0|abnormal
abdomen|0|enteric tube
abdomen|0|sub-diaphragmatic air
abdomen|1|abnormal
abdomen|1|enteric tube
abdomen|1|normal
abdomen|1|sub-diaphragmatic air
abdomen|1|swan-ganz catheter
aortic arch|0|tortuous aorta
aortic arch|1|abnormal
aortic arch|1|intra-aortic balloon pump
aortic arch|1|mediastinal widening
aortic arch|1|normal
aortic arch|1|tortuous aorta
aortic arch|1|vascular calcification
cardiac silhouette|0|abnormal
cardiac silhouette|0|cardiac pacer and wires
cardiac silhouette|0|enlarged cardiac silhouette
cardiac silhouette|0|fluid overload/heart failure
cardiac silhouette|0|mediastinal displacement
cardiac silhouette|0|normal
cardiac silhouette|1|abnormal
cardiac silhouette|1|cabg grafts
cardiac silhouette|1|cardiac pacer and wires
cardiac silhouette|1|chest port
cardiac silhouette|1|enlarged cardiac silhouette
cardiac silhouette|1|fluid overload/heart failure
cardiac silhouette|1|hernia
cardiac silhouette|1|ij l

In [17]:
# Evaluation over all the annotated sentences

print('Per sentence level NLP evaluation:')
truepos = set(attributes.annot_id).intersection(set(scene_attr.annot_id))
falseneg = set(attributes.annot_id).difference(set(scene_attr.annot_id))
falsepos = set(scene_attr.annot_id).difference(set(attributes.annot_id))
precision = len(truepos)/(len(truepos)+len(falsepos))
recall = len(truepos)/(len(truepos)+len(falseneg))
print('True positive sent level:', len(truepos))
print('False negative sent level:', len(falseneg))
print('False positive sent level:', len(falsepos))
print('Number of relations:', len(truepos) + len(falseneg))
print('Precision sent level:', precision)
print('Recall sent level:', recall)
print('f1-score sent level:', 2*precision*recall/(precision+recall))

Per sentence level NLP evaluation:
True positive sent level: 20401
False negative sent level: 1192
False positive sent level: 1481
Number of relations: 21593
Precision sent level: 0.9323188008408738
Recall sent level: 0.9447969249293753
f1-score sent level: 0.9385163887291548


# Calculate report level results

In [18]:
## Rolling evaluation to report level

# But there may be multiple sentences describing an attribute at a bbox location
# Usually the last description is most correct (from final report sentences which are ordered after prelim sentences for each study)
scene_attr2 = scene_attr.copy()
scene_attr2['sent_loc'] = [float(x) for x in scene_attr2['sent_loc']]
scene_attr2 = scene_attr.sort_values(by=['image_id','sent_loc','bbox','label_name']).copy()
print(scene_attr2.shape)
scene_attr2 = scene_attr2.drop_duplicates(subset=['image_id','bbox','label_name'],keep='last').reset_index(drop=True).copy()
print(scene_attr2.shape)
scene_attr2['annot_id'] = ['|'.join([str(study),bbox,str(int(rel)),lab]) for study, bbox, rel, lab 
                           in zip(scene_attr2['study_id'],scene_attr2['bbox'],scene_attr2['relation'],scene_attr2['label_name'])]

# ground truth was annotated to extract true statement from each sentence. 
# If location info from current sentence is missing, we tried looking for that in the previous sentences from the same report
# In any case, the same phenomenon as above can exist
# so doing the same post processing here for ground truth
attributes2 = attributes.copy()
attributes2['sent_loc'] = [float(x.split('|')[-1]) for x in attributes2['row_id']]
attributes2['attribute'] = ['|'.join([cat,context,lab]) for cat, context, lab in zip(attributes2['categoryID'],attributes2['context'],attributes2['label_name'])]
attributes2 = attributes2.sort_values(by=['image_id','sent_loc','bbox','label_name']).copy()
print(attributes2.shape)
attributes2 = attributes2.drop_duplicates(subset=['image_id','bbox','label_name'],keep='last').reset_index(drop=True).copy()
print(attributes2.shape)
attributes2['annot_id'] = ['|'.join([str(study),bbox,str(int(rel)),lab]) for study, bbox, rel, lab 
                           in zip(attributes2['study_id'],attributes2['bbox'],attributes2['relation'],attributes2['label_name'])]


(21882, 18)
(16581, 18)
(21594, 16)
(16569, 16)


In [19]:
# sanity checks -- should be 0's
print(len(scene[(scene.relation==0)&(scene.attribute.str.contains('|yes|',regex=False))]))
print(len(attributes2[(attributes2.relation==1)&(attributes2.attribute.str.contains('|no|',regex=False))]))

0
0


In [20]:
print('Per report level NLP evaluation:')
truepos = set(attributes2.annot_id).intersection(set(scene_attr2.annot_id))
falseneg = set(attributes2.annot_id).difference(set(scene_attr2.annot_id))
falsepos = set(scene_attr2.annot_id).difference(set(attributes2.annot_id))
precision = len(truepos)/(len(truepos)+len(falsepos))
recall = len(truepos)/(len(truepos)+len(falseneg))
print('True positive report level:', len(truepos))
print('False negative report level:', len(falseneg))
print('False positive report level:', len(falsepos))
print('Number of relations:', len(truepos) + len(falseneg))
print('Precision report level:', precision)
print('Recall report level:', recall)
print('f1-score report level:', 2*precision*recall/(precision+recall))

Per report level NLP evaluation:
True positive report level: 15564
False negative report level: 1005
False positive report level: 1017
Number of relations: 16569
Precision report level: 0.938664736746879
Recall report level: 0.9393445591164222
f1-score report level: 0.9390045248868778


# Calculate attribute level results

In [21]:
# If only want to look at results for affirmed relations then set to True

affirmed_only = True

label_names = list(set(attributes.label_name))
label_names.sort()

sent_results = {'label_name':[],'freq':[],'TP':[],'FN':[],'FP':[],'precision':[],'recall':[],'f1-score':[]}
rep_results = {'label_name':[],'freq':[],'TP':[],'FN':[],'FP':[],'precision':[],'recall':[],'f1-score':[]}

for lab in label_names:
    # Sent level results table per label
    if affirmed_only:
        dfgt = attributes[(attributes.label_name == lab)&(attributes.context=='yes')].copy()
        dfscene = scene_attr[(scene_attr.label_name==lab)&(scene_attr.context=='yes')].copy()
    else:
        dfgt = attributes[attributes.label_name == lab].copy()
        dfscene = scene_attr[scene_attr.label_name==lab].copy()
    tp = set(dfgt.annot_id).intersection(dfscene.annot_id)
    fn = set(dfgt.annot_id).difference(dfscene.annot_id)
    fp = set(dfscene.annot_id).difference(dfgt.annot_id)
    if len(tp)+len(fp) > 0:
        prec = len(tp)/(len(tp)+len(fp))
    else:
        prec = 'NaN'
    if len(tp)+len(fn) > 0:
        rec = len(tp)/(len(tp)+len(fn))
    else:
        rec = 'NaN'
    if (prec != 'NaN') & (rec != 'NaN'):
        if prec + rec > 0:
            f1 = 2*prec*rec / (prec + rec)
        else:
            f1 = 0
    else:
        f1 = 0
    sent_results['label_name'].append(lab)
    sent_results['freq'].append(len(dfgt))
    sent_results['TP'].append(len(tp))
    sent_results['FN'].append(len(fn))
    sent_results['FP'].append(len(fp))
    sent_results['precision'].append(prec)
    sent_results['recall'].append(rec)
    sent_results['f1-score'].append(f1)
    
    # Report level results table per label
    if affirmed_only:
        rep_gt = set(attributes2[(attributes2.label_name==lab)&(attributes2.context=='yes')].annot_id)
        rep_scene = set(scene_attr2[(scene_attr2.label_name==lab)&(scene_attr2.context=='yes')].annot_id)
    else:
        rep_gt = set(attributes2[attributes2.label_name==lab].annot_id)
        rep_scene = set(scene_attr2[scene_attr2.label_name==lab].annot_id)
    truepos = rep_gt.intersection(rep_scene)
    falseneg = rep_gt.difference(rep_scene)
    falsepos = rep_scene.difference(rep_gt)
    if len(truepos)+len(falsepos) > 0:
        precision = len(truepos)/(len(truepos)+len(falsepos))
    else:
        precision = 'NaN'
    if len(truepos)+len(falseneg) >0:
        recall = len(truepos)/(len(truepos)+len(falseneg))
    else:
        recall = 'NaN'
    if (precision != 'NaN') & (recall != 'NaN'):
        if precision + recall >0:
            f1score = 2*precision*recall / (precision + recall)
        else:
            f1score = 0
    else:
        f1score = 0
    rep_results['label_name'].append(lab)
    rep_results['freq'].append(len(rep_gt))
    rep_results['TP'].append(len(truepos))
    rep_results['FN'].append(len(falseneg))
    rep_results['FP'].append(len(falsepos))
    rep_results['precision'].append(precision)
    rep_results['recall'].append(recall)
    rep_results['f1-score'].append(f1score)
    
sent_results = pd.DataFrame(sent_results)
sent_results = sent_results.sort_values(by=['freq','f1-score'],ascending=False).reset_index(drop=True).copy()
print(sent_results.shape)
rep_results = pd.DataFrame(rep_results)
rep_results = rep_results.sort_values(by=['freq','f1-score'],ascending=False).reset_index(drop=True).copy()
print(rep_results.shape)

(70, 8)
(70, 8)


In [22]:
# Attribute level results (across all bbox locations) at the sentence level
sent_results

,label_name,freq,TP,FN,FP,precision,recall,f1-score
0,abnormal,3419,3192,227,234,0.931699,0.933606,0.932652
1,lung opacity,2441,2299,142,161,0.934553,0.941827,0.938176
2,normal,2441,2197,244,264,0.892727,0.900041,0.896369
3,atelectasis,685,648,37,24,0.964286,0.945985,0.955048
4,pleural effusion,543,504,39,35,0.935065,0.928177,0.931608
...,...,...,...,...,...,...,...,...
65,tracheostomy tube,3,3,0,0,1.000000,1.000000,1.000000
66,mediastinal drain,2,2,0,0,1.000000,1.000000,1.000000
67,bone lesion,2,2,0,3,0.400000,1.000000,0.571429
68,cyst/bullae,1,1,0,0,1.000000,1.000000,1.000000


In [23]:
# Attribute level results (across all bbox locations) at the report level
rep_results

,label_name,freq,TP,FN,FP,precision,recall,f1-score
0,abnormal,1940,1805,135,183,0.907948,0.930412,0.919043
1,normal,1908,1695,213,159,0.914239,0.888365,0.901116
2,lung opacity,1345,1231,114,74,0.943295,0.915242,0.929057
3,atelectasis,501,475,26,23,0.953815,0.948104,0.950951
4,pleural effusion,345,317,28,34,0.903134,0.918841,0.910920
...,...,...,...,...,...,...,...,...
65,bone lesion,2,1,1,2,0.333333,0.500000,0.400000
66,cyst/bullae,1,1,0,0,1.000000,1.000000,1.000000
67,mediastinal drain,1,1,0,0,1.000000,1.000000,1.000000
68,pericardial effusion,1,1,0,0,1.000000,1.000000,1.000000


In [24]:
# For affirmed only

# To report in paper -- or too cluttered
# Almost all negated relations are correct and the errors come from failed negation or from wrong location assignment for affirmed attributes
if affirmed_only:
    labels4paper = rep_results.copy()
    labels4paper['attribute'] = ['|'.join([scene[scene.label_name==x].categoryID.tolist()[0],'yes',x]) for x in labels4paper['label_name']]
    keep = ['attribute','freq','TP','FN','FP','precision','recall','f1-score']
    labels4paper[keep].to_csv('affirmed_attributes_eval4paper.csv',encoding='utf-8',index=False)

In [25]:
labels4paper.head(50)

,label_name,freq,TP,FN,FP,precision,recall,f1-score,attribute
0,abnormal,1940,1805,135,183,0.907948,0.930412,0.919043,nlp|yes|abnormal
1,normal,1908,1695,213,159,0.914239,0.888365,0.901116,nlp|yes|normal
2,lung opacity,1345,1231,114,74,0.943295,0.915242,0.929057,anatomicalfinding|yes|lung opacity
3,atelectasis,501,475,26,23,0.953815,0.948104,0.950951,anatomicalfinding|yes|atelectasis
4,pleural effusion,345,317,28,34,0.903134,0.918841,0.910920,anatomicalfinding|yes|pleural effusion
5,vascular congestion,295,266,29,8,0.970803,0.901695,0.934974,anatomicalfinding|yes|vascular congestion
6,pulmonary edema/hazy opacity,279,264,15,30,0.897959,0.946237,0.921466,anatomicalfinding|yes|pulmonary edema/hazy opa...
7,low lung volumes,231,219,12,9,0.960526,0.948052,0.954248,technicalassessment|yes|low lung volumes
8,pneumonia,215,200,15,22,0.900901,0.930233,0.915332,disease|yes|pneumonia
9,enteric tube,168,150,18,3,0.980392,0.892857,0.934579,tubesandlines|yes|enteric tube


In [26]:
if affirmed_only:
    sent_results.to_csv('affirmed_attributes_across_bboxes_sentence_level_results.csv',encoding='utf-8',index=False)
    rep_results.to_csv('affirmed_attributes_across_bboxes_report_level_results.csv',encoding='utf-8',index=False)
else:
    sent_results.to_csv('attributes_across_bboxes_sentence_level_results.csv',encoding='utf-8',index=False)
    rep_results.to_csv('attributes_across_bboxes_report_level_results.csv',encoding='utf-8',index=False)

# Calculate bbox level results

In [27]:
# If only want to look at results for affirmed relations then set to True
affirmed_only = True

bbox_names = list(set(attributes.bbox))
bbox_names.sort()

bboxsent_results = {'bbox_name':[],'freq':[],'TP':[],'FN':[],'FP':[],'precision':[],'recall':[],'f1-score':[]}
bboxrep_results = {'bbox_name':[],'freq':[],'TP':[],'FN':[],'FP':[],'precision':[],'recall':[],'f1-score':[]}

for lab in bbox_names:
    # Sent level results table per label
    if affirmed_only:
        dfgt = attributes[(attributes.bbox == lab)&(attributes.context=='yes')].copy()
        dfscene = scene_attr[(scene_attr.bbox==lab)&(scene_attr.context=='yes')].copy()
    else:
        dfgt = attributes[attributes.bbox == lab].copy()
        dfscene = scene_attr[scene_attr.bbox==lab].copy()
    tp = set(dfgt.annot_id).intersection(dfscene.annot_id)
    fn = set(dfgt.annot_id).difference(dfscene.annot_id)
    fp = set(dfscene.annot_id).difference(dfgt.annot_id)
    if len(tp)+len(fp) > 0:
        prec = len(tp)/(len(tp)+len(fp))
    else:
        prec = 'NaN'
    if len(tp)+len(fn) > 0:
        rec = len(tp)/(len(tp)+len(fn))
    else:
        rec = 'NaN'
    if (prec != 'NaN') & (rec != 'NaN'):
        if prec + rec > 0:
            f1 = 2*prec*rec / (prec + rec)
        else:
            f1 = 0
    else:
        f1 = 0
    bboxsent_results['bbox_name'].append(lab)
    bboxsent_results['freq'].append(len(dfgt))
    bboxsent_results['TP'].append(len(tp))
    bboxsent_results['FN'].append(len(fn))
    bboxsent_results['FP'].append(len(fp))
    bboxsent_results['precision'].append(prec)
    bboxsent_results['recall'].append(rec)
    bboxsent_results['f1-score'].append(f1)
    
    # Report level results table per label
    if affirmed_only:
        rep_gt = set(attributes2[(attributes2.bbox==lab)&(attributes2.context=='yes')].annot_id)
        rep_scene = set(scene_attr2[(scene_attr2.bbox==lab)&(scene_attr2.context=='yes')].annot_id)
    else:
        rep_gt = set(attributes2[attributes2.bbox==lab].annot_id)
        rep_scene = set(scene_attr2[scene_attr2.bbox==lab].annot_id)
    truepos = rep_gt.intersection(rep_scene)
    falseneg = rep_gt.difference(rep_scene)
    falsepos = rep_scene.difference(rep_gt)
    if len(truepos)+len(falsepos) > 0:
        precision = len(truepos)/(len(truepos)+len(falsepos))
    else:
        precision = 'NaN'
    if len(truepos)+len(falseneg) >0:
        recall = len(truepos)/(len(truepos)+len(falseneg))
    else:
        recall = 'NaN'
    if (precision != 'NaN') & (recall != 'NaN'):
        if precision + recall >0:
            f1score = 2*precision*recall / (precision + recall)
        else:
            f1score = 0
    else:
        f1score = 0
    bboxrep_results['bbox_name'].append(lab)
    bboxrep_results['freq'].append(len(rep_gt))
    bboxrep_results['TP'].append(len(truepos))
    bboxrep_results['FN'].append(len(falseneg))
    bboxrep_results['FP'].append(len(falsepos))
    bboxrep_results['precision'].append(precision)
    bboxrep_results['recall'].append(recall)
    bboxrep_results['f1-score'].append(f1score)
    
bboxsent_results = pd.DataFrame(bboxsent_results)
bboxsent_results = bboxsent_results.sort_values(by=['freq','f1-score'],ascending=False).reset_index(drop=True).copy()
print(bboxsent_results.shape)
bboxrep_results = pd.DataFrame(bboxrep_results)
bboxrep_results = bboxrep_results.sort_values(by=['freq','f1-score'],ascending=False).reset_index(drop=True).copy()
print(bboxrep_results.shape)

(38, 8)
(38, 8)


In [28]:
bboxsent_results

,bbox_name,freq,TP,FN,FP,precision,recall,f1-score
0,left lung,2546,2459,87,241,0.910741,0.965829,0.937476
1,right lung,2543,2459,84,211,0.920974,0.966968,0.943411
2,right lower lung zone,881,759,122,56,0.931288,0.861521,0.895047
3,left lower lung zone,874,808,66,58,0.933025,0.924485,0.928736
4,cardiac silhouette,867,847,20,19,0.978060,0.976932,0.977496
5,left hilar structures,845,806,39,55,0.936121,0.953846,0.944900
6,right hilar structures,838,797,41,64,0.925668,0.951074,0.938199
7,mediastinum,835,792,43,27,0.967033,0.948503,0.957678
8,right costophrenic angle,438,404,34,28,0.935185,0.922374,0.928736
9,left costophrenic angle,436,411,25,39,0.913333,0.942661,0.927765


In [29]:
bboxrep_results 

,bbox_name,freq,TP,FN,FP,precision,recall,f1-score
0,left lung,1453,1382,71,128,0.915232,0.951136,0.932838
1,right lung,1436,1363,73,110,0.925322,0.949164,0.937092
2,cardiac silhouette,633,617,16,27,0.958075,0.974724,0.966327
3,left lower lung zone,609,568,41,42,0.931148,0.932677,0.931911
4,mediastinum,601,573,28,30,0.950249,0.953411,0.951827
5,right lower lung zone,580,509,71,40,0.927140,0.877586,0.901683
6,right hilar structures,572,541,31,45,0.923208,0.945804,0.934370
7,left hilar structures,571,543,28,37,0.936207,0.950963,0.943527
8,upper mediastinum,359,328,31,11,0.967552,0.913649,0.939828
9,left costophrenic angle,298,276,22,34,0.890323,0.926174,0.907895


In [30]:
if affirmed_only:
    bboxsent_results.to_csv('affirmed_bboxes_across_attributes_sentence_level_results.csv',encoding='utf-8',index=False)
    bboxrep_results.to_csv('affirmed_bboxes_across_attributes_report_level_results.csv',encoding='utf-8',index=False)
else:
    bboxsent_results.to_csv('bboxes_across_attributes_sentence_level_results.csv',encoding='utf-8',index=False)
    bboxrep_results.to_csv('bboxes_across_attributes_report_level_results.csv',encoding='utf-8',index=False)